# Análise de sentimento: classificando tweets

In [1]:
import pandas as pd

O objetivo deste relatório é usar o algoritmo Naive Bayes para classificar tweets como sentimentalmente positivos ou negativos. Desde que esse algoritmo segue uma abordagem de aprendizado supervisionada, precisaremos treinar nosso modelo com uma base de dados já rotulada (tweets rotulados como positivos ou negativos).

[Este repositório](https://github.com/pauloemmilio/dataset) faz um esforço no sentido de criar essa base de dados já rotulada. A ideia utilizada foi de que se um tweet possui os emoticons ':)' ou ':-)', então ele é positivo e será rotulado assim. Se p tweet tem emoticons ':(' ou ':-(', ele será rotulado como negativo. Utilizamos essa base, que carregaremos a seguir.

In [2]:
tweets = pd.read_csv("../data/tweets_labeled.csv", encoding="utf-8", delimiter="\t")

Nossa base tem a seguinte cara:

In [3]:
tweets.head()

,id,text,sentiment
0,1,@caprichOreality Fica assim não miga &lt;3 Tud...,1.0
1,2,Parti me todo a descer a avenida de Gaia com o...,1.0
2,3,Amanhã é dia de dar um trato na palestra para ...,1.0
3,4,@thankovsky @patorebaichado eu também tenho :)...,1.0
4,5,ok. Sim. Aham. Tá. De boa. Vai lá. :) https://...,1.0


In [4]:
from IPython.display import Markdown
Markdown("""Todos os tweets estão em português. Ao todo, temos {n_tweets}""".format(n_tweets=tweets.shape[0]))

Todos os tweets estão em português. Ao todo, temos 57483

### Definindo algumas constantes
(que serão usadas ao longo do código)

In [5]:
COL_TOKENIZED_TEXT = 'tokenized_text'
COL_TEXT = 'text'
POSITIVE = 'positive'
NEGATIVE = 'negative'
COL_PREDICT = 'predict'
COL_LABEL = 'label'

## Pré-processamento e limpeza dos dados

In [6]:
import re
import nltk

def remove_stopwords(text):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    words = [i for i in text.split() if not i in stopwords]
    return (" ".join(words))

def remove_links(text):
    text = re.sub(r"https\S+", "", text)
    return re.sub(r"http\S+", "", text)

def remove_mentions(text):
    return re.sub(r"@\w+", "", text)

def remove_special_chars(text):
    text = re.sub(r'[^\w\s]', ' ', text) # remove special chars
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text_with_no_special_chars = re.sub("\s+", " ", text) #remove all duplicated spaces
    return text_with_no_special_chars

def stemming(text):
    stemmer = nltk.stem.RSLPStemmer()
    words = []
    for word in text.split():
        words.append(stemmer.stem(word))
    return (" ".join(words))

def standardize_text(text):
    text = text.lower()
    text = remove_links(text)
    text = remove_mentions(text)
    text = remove_stopwords(text)
    text = remove_special_chars(text)
    return text

Removendo stopwords, alguns caracteres especiais, links e mentions e alterando para lowercase:

In [7]:
tweets.text = tweets.text.apply(standardize_text)

Há muitos tweets que são retweets (e portanto tweets com textos exatamente iguais). Tenho a hipótese de que uma imensa quantidade de retweets pode diminuir a acurácia do modelo, pois, depois de passados os tweets para uma forma vetorizada, que explicarei mais embaixo, algumas palavras podem ter um peso maior do que deveriam, dada a grande quantidade de textos repetidos (o modelo pode haver overfitting caso haja muitos retweets). Portanto, iremos removendo os retweets:

In [8]:
tweets = tweets.drop_duplicates(COL_TEXT)

In [9]:
Markdown("""Agora reduzimos nossa quantidade de tweets para {n_tweets}""".format(n_tweets=tweets.shape[0]))

Agora reduzimos nossa quantidade de tweets para 52760

In [10]:
tweets.head()

,id,text,sentiment
0,1,fica assim miga lt tudo arranja deus quiser,1.0
1,2,parti todo descer avenida gaia skate,1.0
2,3,amanhã é dia dar trato palestra thedevconf aju...,1.0
3,4,posso sentar vocês,1.0
4,5,ok sim aham tá boa vai lá,1.0


Alterando os valores para a classe de tweets (ex: sentimento positivo deixa de ser representado pelo valor 1 e passar a ser a string 'positive'):

In [11]:
def get_sentiment_category(sentiment_float):
    if sentiment_float == 1.0:
        return POSITIVE
    else:
        return NEGATIVE
    
tweets.sentiment = tweets.sentiment.apply(get_sentiment_category)

## O modelo

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

Iremos usar a implementação da lib [scikit](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB) para o algoritmo Naive Bayes multinomial.

Separando os tweets e suas classes:

In [13]:
tweets_texts = tweets.text.values
print(tweets_texts)

['fica assim miga lt tudo arranja deus quiser '
 'parti todo descer avenida gaia skate '
 'amanhã é dia dar trato palestra thedevconf ajustes finais ' ...
 'tô quase ' 'vcs sao tudo uns fudido ' 'to legal ']


In [14]:
classes = tweets.sentiment.values
print(classes)

['positive' 'positive' 'positive' ... 'negative' 'negative' 'negative']


Agora, vamos treinar o modelo.

In [15]:
vectorizer = CountVectorizer(analyzer="word")
#vectorizer = CountVectorizer(ngram_range = (1, 2))
freq_tweets = vectorizer.fit_transform(tweets_texts)

model = MultinomialNB()
model.fit(freq_tweets, classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

A função fit_transform ajusta o modelo, aprende o vocabulário, e transforma os dados de treinamento para uma representação vetorizada com frequêcia das palavras (_bag of words_).

Exemplo dessa representação vetorizada seria:

id; tweet  
1;  "Este é um tweet positivo"  
2 ; "Este é um tweet negativo"  

A forma vetorizada seria:  

id; Este; é; um; tweet; positivo; negativo; label  
1;  1;    1; 1;  1;     1;        0;        1  
2;  1;    1; 1;  1;     0;        1;        0

## Testando modelo

### Algumas verificações de sanidade..

Nossos tweets que serão usados para testar o modelo:

In [16]:
raw_tests = [
    'Esse governo está no início, vamos ver o que vai dar',
    'Estou muito feliz com o governo de Minas esse ano',
    'O estado de Minas Gerais decretou calamidade financeira!!!',
    'A segurança desse país está deixando a desejar',
    'O governador de Minas é do PT',
    'Estou bastante infeliz ultimamente',
    'Menino, tô muito doente']

Limpando esses tweets, semelhante ao que fizemos com nossa base de dados de treinamento:

In [17]:
tests = list(map(standardize_text, raw_tests))

In [18]:
freq_testes = vectorizer.transform(tests)

Classificando nossos tweets de teste:

In [19]:
classifications = model.predict(freq_testes)

Resultados:

In [20]:
for i, tweet in enumerate(zip(raw_tests, classifications)):
    text = tweet[0]
    classz = tweet[1]
    print("# {} - Tweet: {}. Classified as: {}".format(i+1, text, classz))

# 1 - Tweet: Esse governo está no início, vamos ver o que vai dar. Classified as: positive
# 2 - Tweet: Estou muito feliz com o governo de Minas esse ano. Classified as: positive
# 3 - Tweet: O estado de Minas Gerais decretou calamidade financeira!!!. Classified as: negative
# 4 - Tweet: A segurança desse país está deixando a desejar. Classified as: negative
# 5 - Tweet: O governador de Minas é do PT. Classified as: positive
# 6 - Tweet: Estou bastante infeliz ultimamente. Classified as: negative
# 7 - Tweet: Menino, tô muito doente. Classified as: negative


Para esses tweets usados como testes de sanidade, o modelo parece ter feito sentido. Mas há algumas coisas que poderiam ser melhoradas. Por exemplo, o primeiro tweet poderia ser classificado como neutro. Seria o mais apropriado. Mas como nossos dados de treino só são rotulados ou como positivos ou como negativos, nosso modelo só poderá usar essas duas classes também.

#### Testando com dados reais do Twitter (tweets não usados para treino)

Pesquisei pela hashtag #eleicoies2018 usando a API do Twitter e baixei alguns tweets que importaremos agora.

In [21]:
tweets_test = pd.read_csv("../data/tweets_test.csv", encoding="utf-8", delimiter="\t")

In [22]:
tweets_test.head()

,text
0,RT @augustosnunes: #SanatórioGeral Ciro confes...
1,RT @augustosnunes: #SanatórioGeral Ciro confes...
2,@Abreu_Mateus @AdemirAu Não admito que vc fale...
3,"RT @requiaopmdb: Osmar Dias, ex-governo Dilma,..."
4,RT @folha: Defensor de direitos humanos tinha ...


In [23]:
tweets_test['rawtext'] = tweets_test.text

Retirando os tweets que são retweets (começam com 'RT').

In [24]:
tweets_test = tweets_test.loc[~tweets_test[COL_TEXT].str.contains("RT @")]

Removendo duplicados:

In [25]:
tweets_test = tweets_test.drop_duplicates(COL_TEXT)

Agora iremos selecionar 20 tweets e rotulá-los como positivo ou negativo. A ideia é depois comparar meus rótulos com a classificação dada por nosso modelo.

In [26]:
tweets_test = tweets_test.head(20)

In [27]:
for i, tweet in enumerate(tweets_test.text.head(20)):
    tweet_without_link = re.sub(r"https\S+", "", tweet)
    print("~> {} - {}\n".format(i+1, tweet_without_link))

~> 1 - @Abreu_Mateus @AdemirAu Não admito que vc fale assim com a Dilma. Eu já te disse que eu ❤️ela.

~> 2 - Se você está triste hoje, ouça a Dilma e vai doer a barriga de tanto rir. 😂😂😂😂😂😂 

~> 3 - Morre advogado Hélio Bicudo, autor do pedido de impeachment de Dilma  via @UOLNoticias @UOL

~> 4 - Morre aos 96 anos, em SP, Hélio Bicudo, fundador do PT e autor do pedido de impeachment de Dilma 

~> 5 - @AbmcGrazi @augustosnunes A Dilma é insuperável nos seus standups.

~> 6 - Lula ou Dilma? — eu 

~> 7 - "Não voto em Bolsonaro pq ele só fala bobagem e não entende de nada de governo" afirma o eleitor de Lula e de Dilma. ¬¬

~> 8 - Gente ???? ele literalmente ficou um minuto falando da Dilma só pra n responder a pergunta Auahauhaua 

~> 9 - @arielpalacios @GloboNews pensei que era só tirar a Dilma que ia melhorar! 
#sqn #LulaLivreJa #golpe

~> 10 - @GABRELPlNHElRO O entrevistador Néscio  só faltou dar uma de dilma e dizer que tem que haver diálogo com os bandidos kkkkk

~> 11 - @mvsmotta

Atribuindo os rótulos que achei apropriado:

In [28]:
labels = [POSITIVE, POSITIVE, NEGATIVE, NEGATIVE, POSITIVE, POSITIVE, NEGATIVE, POSITIVE, POSITIVE, POSITIVE, NEGATIVE, POSITIVE, POSITIVE, NEGATIVE, POSITIVE, POSITIVE, NEGATIVE, POSITIVE, NEGATIVE, POSITIVE]
tweets_test[COL_LABEL] = labels

Limpandos os tweets de modo semelhante ao que fiz com os dados treinados:

In [29]:
tweets_test.text = tweets_test.text.apply(standardize_text)

In [30]:
tweets_test.head()

,text,rawtext,label
2,admito vc fale assim dilma disse ela,@Abreu_Mateus @AdemirAu Não admito que vc fale...,positive
5,triste hoje ouça dilma vai doer barriga tanto ...,"Se você está triste hoje, ouça a Dilma e vai d...",positive
6,morre advogado hélio bicudo autor pedido impea...,"Morre advogado Hélio Bicudo, autor do pedido d...",negative
7,morre anos sp hélio bicudo fundador pt autor p...,"Morre aos 96 anos, em SP, Hélio Bicudo, fundad...",negative
8,dilma é insuperável standups,@AbmcGrazi @augustosnunes A Dilma é insuperáve...,positive


Transformando os tweets para a representação vetorizada:

In [31]:
freq_testes = vectorizer.transform(tweets_test.text)

Usando o modelo para classificar os tweets:

In [32]:
tweets_test[COL_PREDICT] = model.predict(freq_testes)

In [33]:
print("Examples of tweets predicted as positive...\n")

positive_classified = tweets_test[ tweets_test[COL_PREDICT] == POSITIVE]
for i, tweet in enumerate(positive_classified.rawtext.head(5)):
    tweet = re.sub("\s+", " ", tweet)
    print("~> {} - {}\n".format(i+1, tweet))

Examples of tweets predicted as positive...

~> 1 - @Abreu_Mateus @AdemirAu Não admito que vc fale assim com a Dilma. Eu já te disse que eu ❤️ela.

~> 2 - Lula ou Dilma? — eu https://t.co/eiojd2MmKr

~> 3 - "Não voto em Bolsonaro pq ele só fala bobagem e não entende de nada de governo" afirma o eleitor de Lula e de Dilma. ¬¬

~> 4 - Gente ???? ele literalmente ficou um minuto falando da Dilma só pra n responder a pergunta Auahauhaua https://t.co/hgwxcYTIsR

~> 5 - @GABRELPlNHElRO O entrevistador Néscio só faltou dar uma de dilma e dizer que tem que haver diálogo com os bandidos kkkkk



In [34]:
print("\n\nExamples of tweets predicted as negative...\n")

negative_classified = tweets_test[ tweets_test[COL_PREDICT] == NEGATIVE]
for i, tweet in enumerate(negative_classified.rawtext.head(5)):
    tweet = re.sub("\s+", " ", tweet)
    tweet = re.sub("\s+", " ", tweet)
    print("~> {} - {}\n".format(i+1, tweet))



Examples of tweets predicted as negative...

~> 1 - Se você está triste hoje, ouça a Dilma e vai doer a barriga de tanto rir. 😂😂😂😂😂😂 https://t.co/rJHjTgHilb

~> 2 - Morre advogado Hélio Bicudo, autor do pedido de impeachment de Dilma https://t.co/RX3H5mI0np via @UOLNoticias @UOL

~> 3 - Morre aos 96 anos, em SP, Hélio Bicudo, fundador do PT e autor do pedido de impeachment de Dilma https://t.co/vGR7bGZRpA

~> 4 - @AbmcGrazi @augustosnunes A Dilma é insuperável nos seus standups.

~> 5 - @arielpalacios @GloboNews pensei que era só tirar a Dilma que ia melhorar! #sqn #LulaLivreJa #golpe



Vendo quatos dos 20 tweets de teste que rotulei foram classificados com o mesmo rótulo que dei:

In [35]:
n_matches = tweets_test.loc[tweets_test[COL_LABEL] == tweets_test[COL_PREDICT]].shape[0]
print(n_matches)

12


Os resultados de modo geral não parecem ser muito satisfatórios. Vale lembrar também que eu posso ter errado também ao rotular os tweets.   
Uma coisa que devemos ter em mente também é que os dados usados para treino rotularam de modo muito simplista o sentimento do tweet (se o tweet possui um ':)' ele é positivo; se possui um ':(', é negativo). 

## Outro modo de avaliar o modelo...

A função **cross_val_predict** (validação cruzada do modelo) divide os dados do modelo em 10 partes, treina o modelo com nove e testa com uma.

In [36]:
results = cross_val_predict(model, freq_tweets, classes, cv = 10)

In [37]:
accuracy = metrics.accuracy_score(classes, results)
print("A acurácia do modelo é de {0:.2f}".format(accuracy))

A acurácia do modelo é de 0.74


In [38]:
sentiments = [POSITIVE, NEGATIVE]
print(metrics.classification_report(classes, results, sentiments))

             precision    recall  f1-score   support

   positive       0.77      0.70      0.73     27104
   negative       0.71      0.78      0.74     25656

avg / total       0.74      0.74      0.74     52760



### Outros comentários...

Uma coisa que identifiquei enquanto construia meu modelo é que usar stemming ou bigrans não influenciou na acurácia do modelo. Testei algumas modificações, mas as perdas ou ganhos não foram consideráveis.